In [ ]:
!pip install datasets
!pip install jsonlines
!pip install bert_score
!pip install git+https://github.com/google-research/bleurt.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 40.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 51.8 MB/s eta 0:00:00
  Cloning https://github.com/google-research/bleurt.git to /tmp/pip-req-build-weld7sgg
  Running command git clone --filter=blob:none --quiet https://github.com/google-research/bleurt.git /tmp/pip-req-build-weld7sgg
  Resolved https://github.com/google-research/bleurt.git to co

In [ ]:
#LLAMA
import numpy as np
import csv
import random
import datasets
filename_test = "esnli_test.csv"
filename_train = "esnli_train.csv"

with open(filename_test, encoding='utf8', newline='') as file:
    csvreader = csv.reader(file)
    data_for_test = [row for row in csvreader]
    data_for_test = np.array(data_for_test[1:])


#Set up Seed and extract inputs from files

np.random.seed(3)
np.random.shuffle(data_for_test)
test_data_label = data_for_test[:,1]
test_data_prem = data_for_test[:,2]
test_data_hyp = data_for_test[:,3]
test_data_exp = data_for_test[:,4]

random.seed(1)
prompt_index = random.sample(range(test_data_exp.shape[0]),400)

test_dict = {}

count = 0
index = 0
label = ""
exp = ""
flag = 0
accuracy = 0
accuracy2 = 0
L = []
bertscores = []
bertscore_metric = datasets.load_metric("bertscore")

bleurtscore_metric = datasets.load_metric("bleurt","BLEURT-20")
meteor_metric = datasets.load_metric("meteor")
bleurtscores = []
meteorscores = []
with open('llama-esnli.out', encoding='utf8', newline='') as file:
    for line in file:
        if("========" in line[:-1] and flag == 3):
            test_dict[test_data_prem[index]+"#####"+test_data_hyp[index]] = label+"#####"+exp
            l = []
            l.extend([test_data_prem[index],test_data_hyp[index],test_data_label[index],test_data_exp[index], label, exp])
            L.append(l)
            true_exp = test_data_exp[index]
            pred_exp = exp
            if (label == test_data_label[index]):
                accuracy += 1
                if(pred_exp != ''):
                    score = 0
                    score = max(score,bertscore_metric.compute(predictions=[pred_exp], references=[true_exp], lang="en")["f1"][0]*100)
                    bertscores.append(score)
                    score = bleurtscore_metric.compute(predictions=[pred_exp], references=[true_exp])['scores'][0]*100
                    bleurtscores.append(score)
                    score = meteor_metric.compute(predictions=[pred_exp], references=[true_exp])['meteor']*100
                    meteorscores.append(score)

            else:
                bertscores.append(0)
                bleurtscores.append(0)
                meteorscores.append(0)
            flag = 0
            count+=1
        if(line[:-1].isnumeric()):
            index = int(line[:-1])
            flag = 1
        elif ('Reason: ' in line and flag == 1):
            exp = line.split('Reason: ')[1][:-1]
            flag = 2
        elif ('Label: contradiction' in line and flag == 2):
            label = line.split('Label: ')[1][:-1]
            flag = 3
        elif ('Label: entailment' in line and flag == 2):
            label = line.split('Label: ')[1][:-1]
            flag = 3
        elif ('Label: neutral' in line and flag == 2):
            label = line.split('Label: ')[1][:-1]
            flag = 3

import csv
with open('llama_esnli.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Premise","Hypothesis","True Label","Gold exp","Pred Label","Pred exp"])
    for l in L:
        writer.writerow(l)
L = []
file = open('bert_scores_llama_esnli.txt','w')
for score in bertscores:
    file.write(str(score)+"\n")
file.close()
bertscores = []
file = open('bleurt_scores_llama_esnli.txt','w')
for score in bleurtscores:
    file.write(str(score)+"\n")
file.close()
bleurtscores = []
file = open('meteor_scores_llama_esnli.txt','w')
for score in meteorscores:
    file.write(str(score)+"\n")
file.close()
meteorscores = []
print(accuracy/count)
print(accuracy2/count)


<ipython-input-2-1b00cedad8d7>:38: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  bertscore_metric = datasets.load_metric("bertscore")


Computing checksums: 100%|##########| 1/1 [00:05<00:00,  5.88s/it]

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


0.49222797927461137
0.0


In [ ]:
#LLAMA2

import numpy as np
import csv
import random

filename_test = "esnli_test.csv"
filename_train = "esnli_train.csv"

with open(filename_test, encoding='utf8', newline='') as file:
    csvreader = csv.reader(file)
    data_for_test = [row for row in csvreader]
    data_for_test = np.array(data_for_test[1:])


#Set up Seed and extract inputs from files

np.random.seed(3)
np.random.shuffle(data_for_test)
test_data_label = data_for_test[:,1]
test_data_prem = data_for_test[:,2]
test_data_hyp = data_for_test[:,3]
test_data_exp = data_for_test[:,4]

random.seed(1)
prompt_index = random.sample(range(test_data_exp.shape[0]),400)

test_dict = {}

count = 0
index = 0
label = ""
exp = ""
flag = 0
accuracy = 0
accuracy2 = 0
L = ['contradiction','neutral']
with open('llama-2-esnli.out', encoding='utf8', newline='') as file:
    for line in file:
        if("========" in line[:-1] and flag == 3):
            test_dict[test_data_prem[index]+"#####"+test_data_hyp[index]] = label+"#####"+exp
            l = []
            l.extend([test_data_prem[index],test_data_hyp[index],test_data_label[index],test_data_exp[index], label, exp])
            L.append(l)
            true_exp = test_data_exp[index]
            pred_exp = exp
            if (label == test_data_label[index]):
                accuracy += 1
                if(pred_exp != ''):
                    score = 0
                    score = max(score,bertscore_metric.compute(predictions=[pred_exp], references=[true_exp], lang="en")["f1"][0]*100)
                    bertscores.append(score)
                    score = bleurtscore_metric.compute(predictions=[pred_exp], references=[true_exp])['scores'][0]*100
                    bleurtscores.append(score)
                    score = meteor_metric.compute(predictions=[pred_exp], references=[true_exp])['meteor']*100
                    meteorscores.append(score)

            else:
                bertscores.append(0)
                bleurtscores.append(0)
                meteorscores.append(0)
            flag = 0
            count+=1
        if(line[:-1].isnumeric()):
            index = int(line[:-1])
            flag = 1
        elif ('Reason: ' in line and flag == 1):
            exp = line.split('Reason: ')[1][:-1]
            flag = 2
        elif ('Label: contradiction' in line and flag == 2):
            label = line.split('Label: ')[1][:-1]
            flag = 3
        elif ('Label: entailment' in line and flag == 2):
            label = line.split('Label: ')[1][:-1]
            flag = 3
        elif ('Label: neutral' in line and flag == 2):
            label = line.split('Label: ')[1][:-1]
            flag = 3

print(accuracy/count)
print(accuracy2/count)
import csv
with open('llama2_esnli.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Premise","Hypothesis","True Label","Gold exp","Pred Label","Pred exp"])
    for l in L:
        writer.writerow(l)
L = []
file = open('bert_scores_llama2_esnli.txt','w')
for score in bertscores:
    file.write(str(score)+"\n")
file.close()
bertscores = []
file = open('bleurt_scores_llama2_esnli.txt','w')
for score in bleurtscores:
    file.write(str(score)+"\n")
file.close()
bleurtscores = []
file = open('meteor_scores_llama2_esnli.txt','w')
for score in meteorscores:
    file.write(str(score)+"\n")
file.close()
meteorscores = []

0.47
0.0


In [ ]:
#VICUNA

import numpy as np
import csv
import random

filename_test = "esnli_test.csv"
filename_train = "esnli_train.csv"

with open(filename_test, encoding='utf8', newline='') as file:
    csvreader = csv.reader(file)
    data_for_test = [row for row in csvreader]
    data_for_test = np.array(data_for_test[1:])


#Set up Seed and extract inputs from files

np.random.seed(3)
np.random.shuffle(data_for_test)
test_data_label = data_for_test[:,1]
test_data_prem = data_for_test[:,2]
test_data_hyp = data_for_test[:,3]
test_data_exp = data_for_test[:,4]

random.seed(1)
prompt_index = random.sample(range(test_data_exp.shape[0]),400)

test_dict = {}

count = 0
index = 0
label = ""
exp = ""
flag = 0
accuracy2 = 0
accuracy = 0
L = ['contradiction','neutral']
with open('vicuna-esnli.out', encoding='utf8', newline='') as file:
    for line in file:
        if("========" in line[:-1] and flag == 3):
            test_dict[test_data_prem[index]+"#####"+test_data_hyp[index]] = label+"#####"+exp
            l = []
            l.extend([test_data_prem[index],test_data_hyp[index],test_data_label[index],test_data_exp[index], label, exp])
            L.append(l)
            true_exp = test_data_exp[index]
            pred_exp = exp
            if (label == test_data_label[index]):
                accuracy += 1
                if(pred_exp != ''):
                    score = 0
                    score = max(score,bertscore_metric.compute(predictions=[pred_exp], references=[true_exp], lang="en")["f1"][0]*100)
                    bertscores.append(score)
                    score = bleurtscore_metric.compute(predictions=[pred_exp], references=[true_exp])['scores'][0]*100
                    bleurtscores.append(score)
                    score = meteor_metric.compute(predictions=[pred_exp], references=[true_exp])['meteor']*100
                    meteorscores.append(score)

            else:
                bertscores.append(0)
                bleurtscores.append(0)
                meteorscores.append(0)
            flag = 0
            count+=1
        if(line[:-1].isnumeric()):
            index = int(line[:-1])
            flag = 1
        elif ('Reason: ' in line and flag == 1):
            exp = line.split('Reason: ')[1][:-1]
            flag = 2
        elif ('Label: contradiction' in line and flag == 2):
            label = line.split('Label: ')[1][:-1]
            flag = 3
        elif ('Label: entailment' in line and flag == 2):
            label = line.split('Label: ')[1][:-1]
            flag = 3
        elif ('Label: neutral' in line and flag == 2):
            label = line.split('Label: ')[1][:-1]
            flag = 3


print(accuracy/count)
print(accuracy2/count)
import csv
with open('vicuna_esnli.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Premise","Hypothesis","True Label","Gold exp","Pred Label","Pred exp"])
    for l in L:
        writer.writerow(l)
L = []
file = open('bert_scores_vicuna_esnli.txt','w')
for score in bertscores:
    file.write(str(score)+"\n")
file.close()
bertscores = []
file = open('bleurt_scores_vicuna_esnli.txt','w')
for score in bleurtscores:
    file.write(str(score)+"\n")
file.close()
bleurtscores = []
file = open('meteor_scores_vicuna_esnli.txt','w')
for score in meteorscores:
    file.write(str(score)+"\n")
file.close()
meteorscores = []

0.5538057742782152
0.0


In [ ]:
#GPT3.5

import numpy as np
import csv
import random

filename_test = "esnli_test.csv"
filename_train = "esnli_train.csv"

with open(filename_test, encoding='utf8', newline='') as file:
    csvreader = csv.reader(file)
    data_for_test = [row for row in csvreader]
    data_for_test = np.array(data_for_test[1:])


#Set up Seed and extract inputs from files

np.random.seed(3)
np.random.shuffle(data_for_test)
test_data_label = data_for_test[:,1]
test_data_prem = data_for_test[:,2]
test_data_hyp = data_for_test[:,3]
test_data_exp = data_for_test[:,4]

random.seed(1)
prompt_index = random.sample(range(test_data_exp.shape[0]),400)

test_dict = {}

count = 0
index = 0
label = ""
exp = ""
flag = 0
accuracy = 0
accuracy2 = 0
L = ['contradiction','neutral']
with open('gpt35-esnli3.out', encoding='utf8', newline='') as file:
    for line in file:
        if("========" in line[:-1] and flag == 3):
            test_dict[test_data_prem[index]+"#####"+test_data_hyp[index]] = label+"#####"+exp
            l = []
            l.extend([test_data_prem[index],test_data_hyp[index],test_data_label[index],test_data_exp[index], label, exp])
            L.append(l)
            true_exp = test_data_exp[index]
            pred_exp = exp
            if (label == test_data_label[index]):
                accuracy += 1
                if(pred_exp != ''):
                    score = 0
                    score = max(score,bertscore_metric.compute(predictions=[pred_exp], references=[true_exp], lang="en")["f1"][0]*100)
                    bertscores.append(score)
                    score = bleurtscore_metric.compute(predictions=[pred_exp], references=[true_exp])['scores'][0]*100
                    bleurtscores.append(score)
                    score = meteor_metric.compute(predictions=[pred_exp], references=[true_exp])['meteor']*100
                    meteorscores.append(score)

            else:
                bertscores.append(0)
                bleurtscores.append(0)
                meteorscores.append(0)
            flag = 0
            count+=1
        if(line[:-1].isnumeric()):
            index = int(line[:-1])
            flag = 1
        elif ('Reason: ' in line and flag == 1):
            exp = line.split('Reason: ')[1][:-1]
            flag = 2
        elif ('Label: contradiction' in line and flag == 2):
            label = line.split('Label: ')[1][:-1]
            flag = 3
        elif ('Label: entailment' in line and flag == 2):
            label = line.split('Label: ')[1][:-1]
            flag = 3
        elif ('Label: neutral' in line and flag == 2):
            label = line.split('Label: ')[1][:-1]
            flag = 3

print(accuracy/count)
print(accuracy2/count)
import csv
with open('gpt35_esnli.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Premise","Hypothesis","True Label","Gold exp","Pred Label","Pred exp"])
    for l in L:
        writer.writerow(l)
L = []
file = open('bert_scores_gpt35_esnli.txt','w')
for score in bertscores:
    file.write(str(score)+"\n")
file.close()
bertscores = []
file = open('bleurt_scores_gpt35_esnli.txt','w')
for score in bleurtscores:
    file.write(str(score)+"\n")
file.close()
bleurtscores = []
file = open('meteor_scores_gpt35_esnli.txt','w')
for score in meteorscores:
    file.write(str(score)+"\n")
file.close()
meteorscores = []

0.6915254237288135
0.0


In [ ]:
#GPT4

import numpy as np
import csv
import random

filename_test = "esnli_test.csv"
filename_train = "esnli_train.csv"

with open(filename_test, encoding='utf8', newline='') as file:
    csvreader = csv.reader(file)
    data_for_test = [row for row in csvreader]
    data_for_test = np.array(data_for_test[1:])


#Set up Seed and extract inputs from files

np.random.seed(3)
np.random.shuffle(data_for_test)
test_data_label = data_for_test[:,1]
test_data_prem = data_for_test[:,2]
test_data_hyp = data_for_test[:,3]
test_data_exp = data_for_test[:,4]

random.seed(1)
prompt_index = random.sample(range(test_data_exp.shape[0]),400)

test_dict = {}

count = 0
index = 0
label = ""
exp = ""
flag = 0
accuracy = 0
accuracy2 = 0
L = ['contradiction','neutral']
with open('gpt4-esnli2.out', encoding='utf8', newline='') as file:
    for line in file:
        if("========" in line[:-1] and flag == 3):
            test_dict[test_data_prem[index]+"#####"+test_data_hyp[index]] = label+"#####"+exp
            l = []
            l.extend([test_data_prem[index],test_data_hyp[index],test_data_label[index],test_data_exp[index], label, exp])
            L.append(l)
            true_exp = test_data_exp[index]
            pred_exp = exp
            if (label == test_data_label[index]):
                accuracy += 1
                if(pred_exp != ''):
                    score = 0
                    score = max(score,bertscore_metric.compute(predictions=[pred_exp], references=[true_exp], lang="en")["f1"][0]*100)
                    bertscores.append(score)
                    score = bleurtscore_metric.compute(predictions=[pred_exp], references=[true_exp])['scores'][0]*100
                    bleurtscores.append(score)
                    score = meteor_metric.compute(predictions=[pred_exp], references=[true_exp])['meteor']*100
                    meteorscores.append(score)

            else:
                bertscores.append(0)
                bleurtscores.append(0)
                meteorscores.append(0)
            flag = 0
            count+=1
        if(line[:-1].isnumeric()):
            index = int(line[:-1])
            flag = 1
        elif ('Reason: ' in line and flag == 1):
            exp = line.split('Reason: ')[1][:-1]
            flag = 2
        elif ('Label: contradiction' in line and flag == 2):
            label = line.split('Label: ')[1][:-1]
            flag = 3
        elif ('Label: entailment' in line and flag == 2):
            label = line.split('Label: ')[1][:-1]
            flag = 3
        elif ('Label: neutral' in line and flag == 2):
            label = line.split('Label: ')[1][:-1]
            flag = 3


print(accuracy/count)
print(accuracy2/count)

import csv
with open('gpt4_esnli.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Premise","Hypothesis","True Label","Gold exp","Pred Label","Pred exp"])
    for l in L:
        writer.writerow(l)
L = []
file = open('bert_scores_gpt4_esnli.txt','w')
for score in bertscores:
    file.write(str(score)+"\n")
file.close()
bertscores = []
file = open('bleurt_scores_gpt4_esnli.txt','w')
for score in bleurtscores:
    file.write(str(score)+"\n")
file.close()
bleurtscores = []
file = open('meteor_scores_gpt4_esnli.txt','w')
for score in meteorscores:
    file.write(str(score)+"\n")
file.close()
meteorscores = []

0.9189873417721519
0.0


43.39130045218789
44.103032149076945


In [ ]:
scores = []
file = open('bert_scores_llama_esnli.txt','r')
for score in file:
    scores.append(float(score[:-1]))
file.close()
print(np.mean(np.array(scores)))
scores = []
file = open('bert_scores_llama2_esnli.txt','r')
for score in file:
    scores.append(float(score[:-1]))
file.close()
print(np.mean(np.array(scores)))
scores = []
file = open('bert_scores_vicuna_esnli.txt','r')
for score in file:
    scores.append(float(score[:-1]))
file.close()
print(np.mean(np.array(scores)))
scores = []
file = open('bert_scores_gpt35_esnli.txt','r')
for score in file:
    scores.append(float(score[:-1]))
file.close()
print(np.mean(np.array(scores)))
scores = []
file = open('bert_scores_gpt4_esnli.txt','r')
for score in file:
    scores.append(float(score[:-1]))
file.close()
print(np.mean(np.array(scores)))

43.39130045218789
41.45497967302799
48.38281372400719
61.21647911556696
81.91962177240396


In [ ]:
scores = []
file = open('bleurt_scores_llama_esnli.txt','r')
for score in file:
    scores.append(float(score[:-1]))
file.close()
print(np.mean(np.array(scores)))
scores = []
file = open('bleurt_scores_llama2_esnli.txt','r')
for score in file:
    scores.append(float(score[:-1]))
file.close()
print(np.mean(np.array(scores)))
scores = []
file = open('bleurt_scores_vicuna_esnli.txt','r')
for score in file:
    scores.append(float(score[:-1]))
file.close()
print(np.mean(np.array(scores)))
scores = []
file = open('bleurt_scores_gpt35_esnli.txt','r')
for score in file:
    scores.append(float(score[:-1]))
file.close()
print(np.mean(np.array(scores)))
scores = []
file = open('bleurt_scores_gpt4_esnli.txt','r')
for score in file:
    scores.append(float(score[:-1]))
file.close()
print(np.mean(np.array(scores)))

21.058502592571042
19.581543792039156
23.08927174862914
34.71229359254999
46.37225491713874


In [ ]:
scores = []
file = open('meteor_scores_llama_esnli.txt','r')
for score in file:
    scores.append(float(score[:-1]))
file.close()
print(np.mean(np.array(scores)))
scores = []
file = open('meteor_scores_llama2_esnli.txt','r')
for score in file:
    scores.append(float(score[:-1]))
file.close()
print(np.mean(np.array(scores)))
scores = []
file = open('meteor_scores_vicuna_esnli.txt','r')
for score in file:
    scores.append(float(score[:-1]))
file.close()
print(np.mean(np.array(scores)))
scores = []
file = open('meteor_scores_gpt35_esnli.txt','r')
for score in file:
    scores.append(float(score[:-1]))
file.close()
print(np.mean(np.array(scores)))
scores = []
file = open('meteor_scores_gpt4_esnli.txt','r')
for score in file:
    scores.append(float(score[:-1]))
file.close()
print(np.mean(np.array(scores)))

15.921241811948601
11.394267227958089
18.455673925523723
29.561536798717952
35.86718479662337


In [ ]:
!pip install git+https://github.com/google-research/bleurt.git
bleurtscore_metric = datasets.load_metric("bleurt",checkpoint = "BLEURT-20")

  Cloning https://github.com/google-research/bleurt.git to /tmp/pip-req-build-dpzn3nz1
  Running command git clone --filter=blob:none --quiet https://github.com/google-research/bleurt.git /tmp/pip-req-build-dpzn3nz1
  Resolved https://github.com/google-research/bleurt.git to commit cebe7e6f996b40910cfaa520a63db47807e3bf5c
  Preparing metadata (setup.py) ... done


In [ ]:
bleurtscore_metric = datasets.load_metric("bleurt","BLEURT-20")
print(bleurtscore_metric.compute(predictions=["Hello"], references=["Yello"]))

Computing checksums: 100%|##########| 1/1 [00:08<00:00,  8.37s/it]

{'scores': [0.40025025606155396]}


In [ ]:
from bleurt import score

checkpoint = "bleurt/test_checkpoint"
references = ["This is a test."]
candidates = ["This is the test."]

scorer = score.BleurtScorer("")
scores = scorer.score(references=references, candidates=candidates)
assert isinstance(scores, list) and len(scores) == 1
print(scores)

[0.7463243007659912]


In [ ]:

print(bleurtscore_metric.compute(predictions=["Hello"], references=["Hello"]))

{'scores': [0.9043734669685364]}


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


{'meteor': 0.0}


In [ ]:
print(meteor_metric.compute(predictions=["Hello m o"], references=["Hello m o"])['meteor'])

0.9814814814814815


In [ ]:

predictions = ["on the mat sat the cat","It is a guide to action which ensures that the military always obeys the commands of the party"]
references = ["the cat sat on the mat","It is a guide to action that ensures that the military will forever heed Party commands"]
results = meteor_metric.compute(predictions=predictions, references=references)

In [ ]:
print(results)

{'meteor': 0.5972222222222223}


In [ ]:
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 1.3 MB/s eta 0:00:00


In [ ]:
import evaluate
meteor = evaluate.load('meteor')
predictions = ["It is a guide to action which ensures that the military always obeys the commands of the party"]
references = ["It is a guide to action that ensures that the military will forever heed Party commands"]
results = meteor.compute(predictions=predictions, references=references)
print(results)

ImportError: ignored